# USGS Stream Flow Bulk Downloader
AUTH: Nathan T. Stevens  
ORG: Pacific Northwest Seismic Network  
LICENSE: GNU GPLv3  
PURPOSE: This notebook details how to get USGS surface water gage data from the WaterWatch dataservice. 

In [1]:
import pandas as pd
from pathlib import Path
import requests, os
from collections import defaultdict

In [6]:
# Map location of data directory created by 
# USGS_Stream_Gauge_Metadata_Downloader.ipynb
PWD = Path().cwd()
DATADIR = PWD/'USGS_Stream_Gage'
SITE_CSV = DATADIR/'usgs_gage_site_metadata.csv'
batchsize = 10
# Specify time-range for query
t0 = pd.Timestamp('2025-11-01 00:00:00', tz='US/Pacific')
t1 = pd.Timestamp('2025-12-31 23:59:59', tz='US/Pacific')
# Data types
params = ['stage','discharge']

In [7]:
# Load site metadata
df_site = pd.read_csv(SITE_CSV, index_col='id')
display(df_site)

,name,lat,lng,class,url,huc_cd,Date,Status,Class,Discharge (cfs),% normal(median) (%),Stage (adj) (ft),% normal(mean) (%),Stage (ft),Length of record (years)
id,,,,,,,,,,,,,,,
10352500,"USGS 10352500 MCDERMITT CK NR MCDERMITT, NV",41.96655720,-117.83181200,4,https://waterdata.usgs.gov/monitoring-location...,16040201,2025-12-15 12:00:00-08:00,NaN,10-24,3.77,49.93,4547.23,28.54,2.23,74.0
10387110,USGS 10387110 CHEWAUCAN RIVER AT MOUTH NEAR VA...,42.52208056,-120.24945000,0,https://waterdata.usgs.gov/monitoring-location...,171200060506,2025-12-15 12:00:00-08:00,NaN,Not-ranked,NaN,NaN,NaN,NaN,8.52,NaN
10387150,"USGS 10387150 LAKE ABERT NEAR VALLEY FALLS, OR",42.60350000,-120.18730560,0,https://waterdata.usgs.gov/monitoring-location...,17120006,2025-12-15 12:45:00-08:00,NaN,Not-ranked,NaN,NaN,4253.30,NaN,4253.30,NaN
10396000,USGS 10396000 DONNER UND BLITZEN RIVER NR FREN...,42.79083330,-118.86750000,5,https://waterdata.usgs.gov/monitoring-location...,17120003,2025-12-15 12:00:00-08:00,NaN,25-75,50.50,120.24,4262.32,90.19,1.99,94.0
11491450,"USGS 11491450 IRVING CREEK NEAR LENZ, OR",42.95166667,-121.45905560,0,https://waterdata.usgs.gov/monitoring-location...,18010201,2025-12-15 12:30:00-08:00,NaN,Not-ranked,0.92,NaN,4636.71,NaN,19.71,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14378200,"USGS 14378200 ILLINOIS RIVER NEAR AGNESS, OR",42.54487778,-124.05191390,0,https://waterdata.usgs.gov/monitoring-location...,17100311,2025-12-15 12:45:00-08:00,NaN,Not-ranked,698.00,21.61,116.69,8.23,4.69,21.0
14378430,"USGS 14378430 ROGUE RIVER AT HWY 101 BRIDGE, A...",42.42888889,-124.41222220,0,https://waterdata.usgs.gov/monitoring-location...,17100310,2025-12-15 12:40:00-08:00,NaN,Not-ranked,NaN,NaN,0.91,NaN,2.91,NaN
14400000,"USGS 14400000 CHETCO RIVER NEAR BROOKINGS, OR",42.12344278,-124.18731070,3,https://waterdata.usgs.gov/monitoring-location...,17100312,2025-12-15 12:30:00-08:00,NaN,<10,606.00,15.25,49.36,9.84,0.20,54.0


In [ ]:
# Define global parameters
_BASE_URL = 'https://nwis.waterservices.usgs.gov/nwis/iv/?'
# Data product mapping parameters
_PARCD_MAP = {
    'stage': '00065',
    'discharge': '00060',
    'temperature': '00010'
}
# Query key terms
_Q_KEYS = ['sites','agencyCd','startDT','endDT','parameterCd','format']
_UNITS = {'stage': 'ft', 'discharge': 'cfs', 'temperature': 'C'}
# Time shifts (ISO 8601 offsets)
_TZ_shift = {
    'UTC': '+00:00',
    'PST': '-08:00',
    'PDT': '-07:00',
    'MST': '-07:00',
    'MDT': '-06:00',
    'CST': '-06:00',
    'CDT': '-05:00',
    'EST': '-05:00',
    'EDT': '-04:00',
    'AKST': '-09:00',
    'AKDT': '-08:00',
    'HST': '-10:00'
}


In [73]:
# # The USGS instantaneous gauge data query is limited to a maximum of 100 stations per query
# # this block splits out the requested stations into <= 100 batches
# batchsize = int(batchsize)
# if batchsize > 100:
#     batchsize = 100
# elif batchsize < 0:
#     batchsize = 1
# else:
#     pass
# if len(df_site) > batchsize:
#     batches = {_e: df_site.iloc[_e*batchsize: int((_e + 1)*batchsize)] for _e in range((len(df_site)//batchsize) + 1)}
# else:
#     batches = {0: df_site}

# print(f'Split into {len(batches)} batches')
# for bn, _dfs in batches.items():
#     print(f'Batch {bn}: {len(_dfs)} sites')

In [ ]:
for _e, (idx, row) in enumerate(df_site.iterrows()):
    print(f'Processing site {idx} ({_e + 1}/{len(df_site)})')
    site_str = str(idx)
    param_str = ','.join([_PARCD_MAP[_p] for _p in params])
    _url = _BASE_URL + \
        f'sites={site_str}&agencyCd=USGS&' + \
        f'parameterCd={param_str}&' + \
        f'startDT={t0.isoformat()}&endDT={t1.isoformat()}&' + \
        'format=rdb'
    request = requests.get(_url)
    if request.status_code == 400:
        print('status_code: 400 - bad query - skipping')
        continue

    lines = request.text.split('\n')
    hdr = []
    body = defaultdict(list)
    for line in lines:
        if line == '':
            continue
        elif '#' == line[0]:
            hdr.append(line)
        elif 'agency_cd' in line:
            cols = []
            for col in line.split('\t'):
                if any(_k in col for _k in _Q_KEYS):
                    parts = col.split('_')
                    parname = _PARCD_MAP[parts[1]]
                    if parts[-1] == 'cd':
                        cols.append('_'.join([parname, 'cd']))
                    else:
                        cols.append(parname)
                else:
                    cols.append(col)
        elif '5s' == line[:2]:
            fstr = line.split('\t')
            fstr_mapping = dict(zip(cols, fstr))
        else:
            parts = line.split('\t')
            try:
                _tmp = dict(zip(cols, parts))
            except:
                breakpoint()
            # Store all values as-is for now
            for _k, _v in _tmp.items():
                if fstr_mapping[_k][-1] == 'n':
                    try:
                        _v = float(_v)
                    except:
                        _v = float('nan')
                body[_k].append(_v)

    # Create DataFrame
    data = pd.DataFrame(body)

    # Parse datetime and convert to UTC.
    # Some responses include `tz_cd`; others may omit it, so handle both cases.
    if 'tz_cd' in data.columns:
        tz_offset = data['tz_cd'].map(_TZ_shift).fillna('+00:00')
        dt_str = data['datetime'].astype(str).str.strip() + tz_offset
        dt_utc = pd.to_datetime(dt_str, errors='coerce', utc=True)
    else:
        dt_utc = pd.to_datetime(data['datetime'], errors='coerce', utc=True)

    # Drop any rows with empty/unparseable datetimes (can occur in some responses)
    bad_dt = dt_utc.isna()
    if bad_dt.any():
        bad_n = int(bad_dt.sum())
        print(f'Warning: dropping {bad_n} rows with unparseable datetime for site {idx}')
        data = data.loc[~bad_dt].copy()
        dt_utc = dt_utc.loc[~bad_dt]

    if len(dt_utc) == 0:
        print(f'No valid datetime rows for site {idx} - skipping')
        continue

    data['datetime'] = dt_utc

    _rnmapper = {}
    for _col in data.columns:
        if '00060' in _col:
            print(f'{_col} to discharge')
            prefix = 'discharge'
            unit = 'cfs'
        elif '00065' in _col:
            prefix = 'gage_height'
            unit = 'ft'
        elif '00010' in _col:
            prefix = 'temperature'
            unit = 'C'
        else:
            continue

        if '_cd' in _col:
            suffix = 'qual'
        else:
            suffix = unit

        _rnmapper.update({_col: '_'.join([prefix, suffix])})

    data = data.rename(columns=_rnmapper)
    data.index = data.datetime
    data.drop(columns=['datetime', 'tz_cd'], inplace=True, errors='ignore')
    if _e == 0:
        display(hdr)
        display(data)
    savedir = DATADIR/str(idx)
    os.makedirs(savedir, exist_ok=True)
    data.to_csv(savedir/f'{idx}_data.csv', header=True, index=True)
    with open(str(savedir/f'{idx}_header.txt'), 'w') as _f:
        for _h in hdr:
            _f.write(f'{_h}\n')
    # if _e > 1:
    #     break
    # data.to_csv(DATADIR/f'{row.org}_{idx}_data.csv')


Processing site 10352500 (1/743)
104148_00060 to discharge
104148_00060_cd to discharge
104148_00060 to discharge
104148_00060_cd to discharge


/var/folders/j4/3rbzxjb16kv66g_p4wkzq16h0000gn/T/ipykernel_76207/1769854607.py:61: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_utc = pd.to_datetime(dt_str, errors='coerce', utc=True)


['# ---------------------------------- WARNING ----------------------------------------',
 '# Some of the data that you have obtained from this U.S. Geological Survey database may not ',
 "# have received Director's approval.  Any such data values are qualified as provisional and ",
 '# are subject to revision.  Provisional data are released on the condition that neither the ',
 '# USGS nor the United States Government may be held liable for any damages resulting from its use.',
 '#  Go to http://help.waterdata.usgs.gov/policies/provisional-data-statement for more information.',
 '#',
 '# File-format description:  http://help.waterdata.usgs.gov/faq/about-tab-delimited-output',
 '# Automated-retrieval info: http://help.waterdata.usgs.gov/faq/automated-retrievals',
 '#',
 '# Contact:   gs-w_support_nwisweb@usgs.gov',
 '# retrieved: 2025-12-15 16:58:34 -05:00\t(nadww02)',
 '#',
 '# Data for the following 1 site(s) are contained in this file',
 '#    USGS 10352500 MCDERMITT CK NR MCDERMITT

,agency_cd,site_no,discharge_cfs,discharge_qual,gage_height_ft,gage_height_qual
datetime,,,,,,
2025-11-01 06:00:00+00:00,USGS,10352500,4.66,P,2.28,P
2025-11-01 06:15:00+00:00,USGS,10352500,4.66,P,2.28,P
2025-11-01 06:30:00+00:00,USGS,10352500,4.66,P,2.28,P
2025-11-01 06:45:00+00:00,USGS,10352500,4.66,P,2.28,P
2025-11-01 07:00:00+00:00,USGS,10352500,4.66,P,2.28,P
...,...,...,...,...,...,...
2025-12-15 20:00:00+00:00,USGS,10352500,3.77,P,2.23,P
2025-12-15 20:15:00+00:00,USGS,10352500,3.77,P,2.23,P
2025-12-15 20:30:00+00:00,USGS,10352500,3.62,P,2.22,P


Processing site 10387110 (2/743)


...

150542_00060 to discharge
150542_00060_cd to discharge
Processing site 12013500 (34/743)


/var/folders/j4/3rbzxjb16kv66g_p4wkzq16h0000gn/T/ipykernel_76207/1769854607.py:61: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_utc = pd.to_datetime(dt_str, errors='coerce', utc=True)


150544_00060 to discharge
150544_00060_cd to discharge
Processing site 12020000 (35/743)


/var/folders/j4/3rbzxjb16kv66g_p4wkzq16h0000gn/T/ipykernel_76207/1769854607.py:61: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_utc = pd.to_datetime(dt_str, errors='coerce', utc=True)


150552_00060 to discharge
150552_00060_cd to discharge
Processing site 12020525 (36/743)
